In [0]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql import functions as F

In [0]:
# Initialize Spark Session
spark = SparkSession.builder.appName('DeltaLakePipeline').getOrCreate()

In [0]:
# Paths for Delta Tables
BRONZE_PATH = '/mnt/data/bronze'
SILVER_PATH = '/mnt/data/silver'
GOLD_PATH = '/mnt/data/gold'

In [0]:
# Ingesting Raw Data (Bronze Layer)
raw_data = [(1, 'Alice', '2025-03-20', 300.0), (2, 'Bob', '2025-03-20', 150.0)]
columns = ['user_id', 'name', 'transaction_date', 'amount']



In [0]:
bronze_df = spark.createDataFrame(raw_data, columns)
bronze_df.write.format('delta').mode('overwrite').save(BRONZE_PATH)
bronze_df.show()

+-------+-----+----------------+------+
|user_id| name|transaction_date|amount|
+-------+-----+----------------+------+
|      1|Alice|      2025-03-20| 300.0|
|      2|  Bob|      2025-03-20| 150.0|
+-------+-----+----------------+------+



In [0]:
# Read Bronze Data
bronze_table = spark.read.format('delta').load(BRONZE_PATH)

In [0]:
# Data Cleaning & Transformation (Silver Layer)
silver_df = bronze_table.filter("amount > 100")  # Removing invalid transactions
silver_df.write.format('delta').mode('overwrite').save(SILVER_PATH)


In [0]:
# Read Silver Data
silver_table = spark.read.format('delta').load(SILVER_PATH)


In [0]:
# Business Aggregation (Gold Layer)
gold_df = silver_table.groupBy('transaction_date').agg(F.sum("amount").alias("Total_amount"))
gold_df.write.format('delta').mode('overwrite').save(GOLD_PATH)

In [0]:
# Read Gold Data
final_table = spark.read.format('delta').load(GOLD_PATH)


In [0]:
# Display the result
final_table.show()

+----------------+------------+
|transaction_date|Total_amount|
+----------------+------------+
|      2025-03-20|       450.0|
+----------------+------------+



In [0]:
# Initialize Spark Session with Delta Lake support
spark = SparkSession.builder.appName("MedallionPipeline").getOrCreate()

In [0]:
# Define paths for each layer
bronze_path = "/mnt/delta/bronze"
silver_path = "/mnt/delta/silver"
gold_path = "/mnt/delta/gold"

In [0]:
# Ingest raw data (Bronze Layer)
raw_data = spark.read.option("inferSchema","True").option("header", "True").csv("/FileStore/tables/employees.csv")
raw_data.write.format("delta").mode("overwrite").option("mergeSchema", "True").save(bronze_path)
raw_data.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [0]:
# Bronze to Silver Transformation (Data Cleaning)
bronze_df = spark.read.format("delta").load(bronze_path)
silver_df = bronze_df.filter('SALARY>6000')

In [0]:
silver_df.write.format("delta").mode("overwrite").save(silver_path)


In [0]:
# Silver to Gold Transformation (Aggregation & Curation)
silver_df = spark.read.format("delta").load(silver_path)
gold_df = silver_df.groupBy("JOB_ID").agg(F.count("*").alias("record_count"), F.sum("SALARY").alias("TOTAL_SALARY"),F.avg("SALARY").alias("AVG_SALARY"))


In [0]:
gold_df.write.format("delta").mode("overwrite").option("mergeSchema", "True").save(gold_path)

In [0]:
gold_df.show()

+----------+------------+------------+----------+
|    JOB_ID|record_count|TOTAL_SALARY|AVG_SALARY|
+----------+------------+------------+----------+
|FI_ACCOUNT|           5|       39600|    7920.0|
|    MK_MAN|           1|       13000|   13000.0|
|   IT_PROG|           1|        9000|    9000.0|
|    FI_MGR|           1|       12008|   12008.0|
|AC_ACCOUNT|           1|        8300|    8300.0|
|    HR_REP|           1|        6500|    6500.0|
|    AC_MGR|           1|       12008|   12008.0|
|    PR_REP|           1|       10000|   10000.0|
|    ST_MAN|           4|       30600|    7650.0|
|    PU_MAN|           1|       11000|   11000.0|
|   AD_PRES|           1|       24000|   24000.0|
|     AD_VP|           2|       34000|   17000.0|
+----------+------------+------------+----------+

